In [4]:
import pandas as pd
df = pd.read_pickle('../datasets/brquad-gte-dev-v2.0_embedded_with_document_names_and_answers.pkl')
df.rename(columns={'document name': 'document_name', 'context': 'chunk_text'}, inplace=True)
df_filtered = df.loc[df['answer'] != 'N/A']  ##remover onde a resposta é N/A
print(df_filtered.head())

                                          chunk_text  \
0  Os normandos (normandos: nourmands; francês: n...   
1  Os normandos (normandos: nourmands; francês: n...   
2  Os normandos (normandos: nourmands; francês: n...   
3  Os normandos (normandos: nourmands; francês: n...   
4  Os normandos (normandos: nourmands; francês: n...   

                                               query  \
0           Em que país a Normandia está localizada?   
1          Quando os normandos estavam na Normandia?   
2         De quais países os nórdicos se originaram?   
3                          Quem era o líder nórdico?   
4  Em que século os normandos ganharam sua identi...   

                                           embedding document_name  \
0  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
1  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
2  [0.02337769791483879, 0.06848494708538055, -0....       Normans   
3  [0.02337769791483879, 0.06848494708538055, 

In [2]:
import time
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from openai_utils import embed_with_openai, ask_with_openai
df_filtered =  pd.read_pickle('hotpot_qa_modificado_validation.pkl')

# Configuração inicial: definindo o número de contextos a serem considerados
n_contexts = 5

def find_closest_contexts(question_embedding, context_embeddings, n=n_contexts):
    distances = cdist(question_embedding, context_embeddings, 'euclidean').flatten()
    closest_indices = np.argsort(distances)[:n]
    return closest_indices

def generate_rag_answer(row, n=n_contexts):
    start_time = time.time()
    question_embedding = np.array(embed_with_openai(row['query'])).reshape(1, -1)
    context_embeddings = np.vstack(df_filtered['embedding'].values)
    closest_indices = find_closest_contexts(question_embedding, context_embeddings, n)
    closest_contexts = df_filtered.iloc[closest_indices]['chunk_text'].tolist()
    answer = ask_with_openai(question=row['query'], context=closest_contexts)
    end_time = time.time()
    execution_time = end_time - start_time
    return answer, execution_time

# Inicia a contagem do tempo total
total_start_time = time.time()

# Aplicando a função modificada

#apply the function for the first 50 rows
results = df_filtered.head(250).apply(generate_rag_answer, axis=1)
#results = df_filtered.apply(generate_rag_answer, axis=1)
#df_filtered['rag_answer'], df_filtered['execution_time'] = zip(*results)
df_filtered['rag_answer'] = results.apply(lambda x: x[0])



# Calcula o tempo total gasto
total_end_time = time.time()
total_execution_time = total_end_time - total_start_time

print(f"Total execution time: {total_execution_time} seconds")

file_name = f"RAG_hotspotqa200_____n-{n_contexts}_____total-time-{(total_execution_time)}s.pkl"

# Salva o DataFrame atualizado
df_filtered.to_pickle(file_name)

# Formatando o nome do arquivo para incluir o número de contextos e o tempo total de execução



Total execution time: 794.4382059574127 seconds


In [8]:
print(df_filtered.columns)

Index(['chunk_text', 'query', 'document_name', 'answer', 'embedding',
       'cluster_label', 'pca_components', 'rag_answer'],
      dtype='object')
